In [1]:
import sys, os
import numpy as np
import pickle
import tensorflow as tf
import multiprocessing
import glob
from tqdm import tqdm
from waymo_open_dataset.protos import scenario_pb2
from waymo_types import object_type, lane_type, road_line_type, road_edge_type, signal_state, polyline_type

2024-03-18 15:24:04.229494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 15:24:04.347666: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 15:24:04.658084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: “/usr/local/cuda-12.0/lib64:/usr/local/lib/:/opt/ros/humble/opt/rviz_ogre_vendor/lib:/opt/ros/humb

In [8]:
example_file = '/home/erik/raid/datasets/womd/processed_scenarios_training/sample_882d8df85189a2c0.pkl'
#training_infos_file = '/home/erik/raid/datasets/womd/processed_scenarios_training_infos.pkl'
# with open(training_infos_file, 'rb') as f:
#     training_infos = pickle.load(f)
with open(example_file, 'rb') as f:
    example = pickle.load(f)
print(example.keys())
print(example['track_infos']['trajs'].shape)
print(len(example['track_infos']['object_id']))
print(example['map_infos'].keys())
# print(training_infos[0].keys())
print(example['track_infos']['object_type'])
print(type(example['scenario_id']))
print(np.array(example['track_infos']['object_type']) == 'TYPE_VEHICLE')
print(example['track_infos']['trajs'][np.array(example['track_infos']['object_type']) == 'TYPE_VEHICLE', :, :3].shape)

dict_keys(['track_infos', 'dynamic_map_infos', 'map_infos', 'scenario_id', 'timestamps_seconds', 'current_time_index', 'sdc_track_index', 'objects_of_interest', 'tracks_to_predict'])
(80, 91, 10)
80
dict_keys(['lane', 'road_line', 'road_edge', 'stop_sign', 'crosswalk', 'driveway', 'speed_bump', 'all_polylines'])
['TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_PEDESTRIAN', 'TYPE_CYCLIST', 'TYPE_CYCLIST', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEHICLE', 'TYPE_VEH

In [33]:
def check_for_vru(track_infos):
    peds = False
    cyclists = False
    for type in track_infos['object_type']:
        if type == 'TYPE_PEDESTRIAN':
            peds = True
        elif type == 'TYPE_CYCLIST':
            cyclists = True
        if peds and cyclists:
            break
    return peds, cyclists

def check_distance_to_vehicles(track_infos):
    vehicle_trajectories = track_infos['trajs'][np.array(track_infos['object_type']) == 'TYPE_VEHICLE', :, :3]
    distances_per_type = dict()
    distances_per_type['TYPE_PEDESTRIAN'] = np.inf
    distances_per_type['TYPE_CYCLIST'] = np.inf
    for i, type in enumerate(track_infos['object_type']):
        if type == 'TYPE_VEHICLE':
            continue
        elif type == 'TYPE_PEDESTRIAN' or type == 'TYPE_CYCLIST':
            ref_trajectory = track_infos['trajs'][i, :, :3]
            differences = vehicle_trajectories - ref_trajectory[np.newaxis, :, :]
            distances = np.linalg.norm(differences, axis=2)
            masked_distances = np.ma.masked_equal(distances, 0.0)
            # print(distances.shape)
            min_distance_per_agent = np.min(masked_distances, axis=1)
            #print(min_distance_per_agent)
            min_distance = np.min(min_distance_per_agent, axis=0)
            if min_distance < distances_per_type[type]:
                distances_per_type[type] = min_distance
    return distances_per_type



In [34]:
distances = check_distance_to_vehicles(example['track_infos'])
print(distances)

{'TYPE_PEDESTRIAN': 2.2126758, 'TYPE_CYCLIST': 5.85942}


In [19]:
# Iterate through dataset
path_to_womd = '/home/erik/raid/datasets/womd'
scenarios = glob.glob(os.path.join(path_to_womd, 'processed_scenarios_training', '*.pkl'))
vru_infos = dict()
for scenario in scenarios:
    with open(scenario, 'rb') as f:
        scenario_dict = pickle.load(f)
    vru_presence = check_for_vru(scenario_dict['track_infos'])
    vru_infos[scenario_dict['scenario_id']] = dict()
    vru_infos[scenario_dict['scenario_id']]['pedestrians'] = vru_presence[0]
    vru_infos[scenario_dict['scenario_id']]['cyclists'] = vru_presence[1]
if vru_presence[0] or vru_presence[1]:
    distances = check_distance_to_vehicles(scenario_dict['track_infos'])
    vru_infos[scenario_dict['scenario_id']]['distances'] = distances
with open('vru_infos.pkl', 'wb') as f:
    pickle.dump(vru_infos, f)
    

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fd5d3653520>>
Traceback (most recent call last):
  File "/home/erik/venv/mtr_pose/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
